# test: jpath

[jpath](../api/tasks.rst#nornir_tests.plugins.tests.jpath) uses jsonpath in conjunction with assertpy to validate task data.

This module combines the power of jsonpath and assertpy.  In order to use this module it is somewhat necessary to be familiar with both of those.  Links to them can be found below.

* `jsonpath_ng <https://github.com/h2non/jsonpath-ng>`
* `assertpy <https://github.com/assertpy/assertpy>`


Examples:

In [17]:
from nornir import InitNornir
from nornir_napalm.plugins.tasks import napalm_get, napalm_cli

from nornir_tests.plugins.functions import print_result
from nornir_tests.plugins.tasks import wrap_task
from nornir_tests.plugins.tests import jpath

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "inventory/hosts.yaml",
            "group_file": "inventory/groups.yaml",
            "defaults_file": "inventory/defaults.yaml",
        }
    },
    dry_run=True,
)

When running a task, validations usually if done in Nornir can be executed as additional logic implemented in python or with running of actual tasks.  Using `nornir_tests` moves the logic into the task and provides a way to impact the success of a task based on its validations.

In [18]:
results = nr.run(
    wrap_task(napalm_get), getters=['facts'],
    tests=[
        jpath(path='$..os_version', value='4.14.3-2329074.gaatlantarel')
    ]
)

In [19]:
print_result(results, vars=['tests', 'highlit'])
print(results['rtr00'][0].result)

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;path&#39;: &#39;$..os_version&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;4.14.3-2329074.gaatlantarel&#39;}
{&#39;matches&#39;: [&#39;facts.os_version&#39;]}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c0714220&gt;

^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;path&#39;: &#39;$..os_version&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;4.14.3-2329074.gaatlantarel&#39;}
{&#39;matches&#39;: [&#39;facts.os_version&#39;]}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c0714310&gt;

^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
{&#39;facts&#39;: {&#39;uptime&#39;: 151005.57332897186, &#39;vendor&#39;: &#39;Arista&#39;, &#39;os_version&#39;: &#39;4.14.3-2329074.gaatlantarel&#39;, &#39;serial_number&#39;: &#39;SN0123A34AS&#39;, &#39;model&#39;: &#39;vEOS&#39;, &#39;hostname&#39;: &#39;eos-router&#39;, &#39;fqdn&#39;: &#39;eos-router&#39;, &#39;interface_list&#39;: [&#39;Ethernet2&#39;, &#39;Management1&#39;, &#39;Ethernet1&#39;, &#39;Ethernet3&#39;]}}


The first example was pretty simple but the next will have many tests run in validating interface data.  It will also use @ decorator syntax.

In [20]:
@jpath(path='$..ipv6', assertion='contains', value="1::1")
@jpath(path='$.interfaces_ip', assertion='is_length', value=3)
@jpath(path='$..FastEthernet8..prefix_length', value=22)
def get_interface_ips(task):
    return napalm_get(task, getters=['interfaces_ip'])

results = nr.run(get_interface_ips)

In [21]:
print_result(results, vars=['tests', 'highlit'])

get_interface_ips***************************************************************
* rtr00 ** changed : False *****************************************************
vvvv get_interface_ips ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;path&#39;: &#39;$..FastEthernet8..prefix_length&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: 22}
{&#39;matches&#39;: [&#39;interfaces_ip.FastEthernet8.ipv4.10.66.43.169.prefix_length&#39;]}
F JpathRecord - {&#39;assertion&#39;: &#39;is_length&#39;,
 &#39;path&#39;: &#39;$.interfaces_ip&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: 3}
{&#39;exception&#39;: TypeError(&quot;&#39;str&#39; object does not support item assignment&quot;),
 &#39;matches&#39;: [&#39;interfaces_ip&#39;]}
P JpathRecord - {&#39;assertion&#39;: &#39;contains&#39;,
 &#39;path&#39;: &#39;$..ipv6&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;1:

&lt;rich.jupyter.JupyterRenderable at 0x7f61c08d0a90&gt;

^^^^ END get_interface_ips ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv get_interface_ips ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;path&#39;: &#39;$..FastEthernet8..prefix_length&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: 22}
{&#39;matches&#39;: [&#39;interfaces_ip.FastEthernet8.ipv4.10.66.43.170.prefix_length&#39;]}
F JpathRecord - {&#39;assertion&#39;: &#39;is_length&#39;,
 &#39;path&#39;: &#39;$.interfaces_ip&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: 3}
{&#39;exception&#39;: TypeError(&quot;&#39;str&#39; object does not support item assignment&quot;),
 &#39;matches&#39;: [&#39;interfaces_ip&#39;]}
F JpathRecord - {&#39;assertion&#39;: &#39;contains&#39;,
 &#39;path&#39;: &#39;$..ipv6&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;1:

&lt;rich.jupyter.JupyterRenderable at 0x7f61c0b6fd60&gt;

^^^^ END get_interface_ips ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


The next example will show how a task can be set to failed based on the validation that is performed.  If the task has fail_task set to true and ends up with passed=False in the test it will mark the overall task as failed.

In [22]:
@jpath(path='$..connection_state', value="Established", fail_task=True)
@jpath(path='$..remote_as', value='8121')
def check_bgp_neighbors(task):
    return napalm_get(task, getters=['bgp_neighbors_detail'])

results = nr.run(check_bgp_neighbors)
print_result(results, vars=['tests', 'highlit'])

check_bgp_neighbors*************************************************************
* rtr00 ** changed : False *****************************************************
vvvv check_bgp_neighbors ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
F JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;path&#39;: &#39;$..remote_as&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;8121&#39;}
{&#39;exception&#39;: Exception(AssertionError(&#39;Expected &lt;8121&gt; to be equal to &lt;8121&gt;, but was not.&#39;))}
P JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;fail_task&#39;: True,
 &#39;path&#39;: &#39;$..connection_state&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;Established&#39;}
{&#39;matches&#39;: [&#39;bgp_neighbors_detail.global.8121.[0].connection_state&#39;]}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c039f2e0&gt;

^^^^ END check_bgp_neighbors ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv check_bgp_neighbors ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
F JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;path&#39;: &#39;$..remote_as&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;8121&#39;}
{&#39;exception&#39;: Exception(AssertionError(&#39;Expected &lt;8121&gt; to be equal to &lt;8121&gt;, but was not.&#39;))}
F JpathRecord - {&#39;assertion&#39;: &#39;is_equal_to&#39;,
 &#39;fail_task&#39;: True,
 &#39;path&#39;: &#39;$..connection_state&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;Established&#39;}
{&#39;exception&#39;: Exception(AssertionError(&#39;Expected &lt;EstabSync&gt; to be equal to &lt;Established&gt;, but was not.&#39;))}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c039f370&gt;

^^^^ END check_bgp_neighbors ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


The last example will use host_data in order to find some data specific to the host to validate against.  The host_data is anything from inventory perhaps obviously in the data dictionary.

In [23]:
nr.data.reset_failed_hosts()

results = nr.run(
    wrap_task(napalm_get), getters=['interfaces'],
    tests=[
        jpath(path='$.interfaces', assertion='contains', host_data='$.mgmt_port')
    ]
)
print_result(results, vars=['tests', 'highlit'])

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;contains&#39;,
 &#39;host_data&#39;: &#39;$.mgmt_port&#39;,
 &#39;path&#39;: &#39;$.interfaces&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;Management1&#39;}
{&#39;matches&#39;: [&#39;interfaces&#39;]}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c19007c0&gt;

^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* rtr01 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;contains&#39;,
 &#39;host_data&#39;: &#39;$.mgmt_port&#39;,
 &#39;path&#39;: &#39;$.interfaces&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;Management2&#39;}
{&#39;matches&#39;: [&#39;interfaces&#39;]}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c052a970&gt;

^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In these examples 'contains', 'is_equal', and 'is_length' were used for assertions.  Many other possibilities are available from the assertpy module.  Not saying all of them make sense to use or that they all work as expected but they should.  Too many to validate to be honest.  Some other that would certainly work fine would be 'is_true', 'is_empty', etc.

A few more things about how it all works.  The one_of argument isn't always needed but it could be.  If the match was intended to turn up many of something and something like is_equal assertion is used, if one_of is not true then it will fail if all matches don't meet the assertion.  This is kind of confusing and I should prob show an example here.

In [24]:
@jpath(path='$..ipv4', assertion='contains', value="10.66.43.169")
@jpath(path='$..ipv4', assertion='contains', value="10.66.43.169", one_of=True)
def get_interface_ips(task):
    return napalm_get(task, getters=['interfaces_ip'])

rtr00 = nr.filter(name='rtr00')
results = rtr00.run(get_interface_ips)
print_result(results, vars=['tests', 'highlit'])

get_interface_ips***************************************************************
* rtr00 ** changed : False *****************************************************
vvvv get_interface_ips ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
P JpathRecord - {&#39;assertion&#39;: &#39;contains&#39;,
 &#39;one_of&#39;: True,
 &#39;path&#39;: &#39;$..ipv4&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;10.66.43.169&#39;}
{&#39;matches&#39;: [&#39;interfaces_ip.FastEthernet8.ipv4&#39;]}
F JpathRecord - {&#39;assertion&#39;: &#39;contains&#39;,
 &#39;path&#39;: &#39;$..ipv4&#39;,
 &#39;result_attr&#39;: &#39;result&#39;,
 &#39;value&#39;: &#39;10.66.43.169&#39;}
{&#39;exception&#39;: Exception(AssertionError(&quot;Expected &lt;{&#39;192.168.1.1&#39;: {&#39;prefix_length&#39;: 24}}&gt; to contain key &lt;10.66.43.169&gt;, but did not.&quot;)),
 &#39;matches&#39;: [&#39;interfaces_ip.FastEthernet8.ipv4&#39;]}


&lt;rich.jupyter.JupyterRenderable at 0x7f61c0c634c0&gt;

^^^^ END get_interface_ips ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


So what happened is the first test passed as it found a bunch of paths that ended with 'ipv4' and it only needed one of them to contain the value of "10.66.43.169".  The second one failed due to the fact that it wanted all the paths to contain that value and they did not.